In [ ]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import mode
from scipy.integrate import cumulative_trapezoid
from scipy.signal import correlate
import json


import gc # garbage collector for removing large variables from memory instantly 
import importlib #for force updating changed packages 

#import harp
import harp_resources.process
import harp_resources.utils
from harp_resources import process, utils # Reassign to maintain direct references for force updating 
#from sleap import load_and_process as lp

In [ ]:
common_resampled_rate = 1000 #in Hz
event_name = "Apply halt: 2s" #event to align data to, can make into list in the future?

#Cohort 1 visual mismatch THIS
rawdata_path = Path('/Users/rancze/Documents/Data/vestVR/Cohort1/Visual_mismatch_day3/B6J2717-2024-12-10T12-17-03')
data_path = rawdata_path.parent / f"{rawdata_path.name}_processedData/downsampled_data"
save_path = rawdata_path.parent / f"{rawdata_path.name}_processedData"
session_name = "_".join(data_path.parts[-2:])

# Load all parquet files from the directory and assign to individual variables
for file in data_path.glob("*.parquet"):
    var_name = file.stem
    if var_name == "session_settings":
        session_settings = pd.read_parquet(file, engine="pyarrow")
        session_settings["metadata"] = session_settings["metadata"].apply(process.safe_from_json)
        print(f"✅ Loaded {var_name} parquet file with special treatment")
    else:
        globals()[var_name] = pd.read_parquet(file, engine="pyarrow")
        print(f"✅ Loaded {var_name} parquet file")




In [ ]:
session_settings

In [ ]:
# detect events in the photodiode signal and align to the nearest timestamps in downsampled data 
photodiode_halts = process.calculate_photodiode_falling_edges(camera_photodiode_data[['Photodiode']], experiment_events, event_name)
nearest_indices = photometry_tracking_encoder_data.index.get_indexer(photodiode_halts, method='nearest')
photodiode_halts = photometry_tracking_encoder_data.index[nearest_indices]
process.plot_figure_1(photometry_tracking_encoder_data, session_name, save_path, common_resampled_rate, photodiode_halts, save_figure = False, show_figure = True, downsample_factor=50)

In [ ]:
# Force reload the modules
importlib.reload(harp_resources.process)
importlib.reload(harp_resources.utils)
# Reassign after reloading to ensure updated references
process = harp_resources.process
utils = harp_resources.utils

In [ ]:
# Load DataFrame from Parquet
session_settings = pd.read_parquet(data_path / "session_settings.parquet", engine="pyarrow")
# Convert JSON string back to dictionary
session_settings["metadata"] = session_settings["metadata"].apply(json.loads)


In [ ]:
session_settings

In [ ]:
pympler_memory_df = utils.get_pympler_memory_usage(top_n=10)
